In [1]:
import sys
import random
import math
import operator
import os

import csv

In [2]:

import numpy as np  # using numpy package for mathematical operations
from numpy import *
from numpy.linalg import inv  # used for inversing the matrix
import numpy.ma

from random import randint
import math

from pyspark import SparkContext

from operator import add

In [3]:

import re
import string
import pandas as pd
from collections import Counter

In [4]:
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.probability import FreqDist as nF
from nltk.corpus import stopwords
from nltk import bigrams
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.tokenize import word_tokenize

In [5]:
from sklearn.naive_bayes import MultinomialNB

In [6]:
stop_words = []
data_set = []
decision_attributes = []
tokens_re = ""

In [7]:
# sc = SparkContext(appName="Naive-Bayes-scikitlib")
sc

<SparkContext master=yarn appName=PySparkShell>

In [8]:
def load_file(file_path):
    print("Loading the dataset...")
    file_read = open(file_path, "r")
    reader = csv.reader(file_read, dialect='excel')
    next(reader, None)
    for row in reader:
        temp = (row[6], row[9], row[12])
        #temp = (row[0], row[1])
        data_set.append(temp)

In [9]:
#http://s3.amazonaws.com/cloudcomputingprojectfall18khansoti/Hotel_Reviews.csv
load_file("Hotel_Reviews.csv")

Loading the dataset...


In [10]:
train_data_features_reviews = pd.DataFrame(data_set, columns=["Negative_Review", "Positive_Review", "Reviewer_Score"])
target_review = train_data_features_reviews['Reviewer_Score']

train_data_features_reviews['Review'] = train_data_features_reviews['Negative_Review'] + " " + train_data_features_reviews['Positive_Review']

In [11]:
train_data_features_reviews.Reviewer_Score = pd.to_numeric(train_data_features_reviews.Reviewer_Score, errors= 'ignore')
train_data_features_reviews.Reviewer_Score = pd.to_numeric(train_data_features_reviews.Reviewer_Score, errors= 'coerce')

train_data_features_reviews.Reviewer_Score = train_data_features_reviews.Reviewer_Score.astype(int)


In [12]:
# Emoticons string if any for removal
emoticons_str = r"""
    (?:
        [:=;] # Eyes
        [oO\-]? # Nose (optional)
        [D\)\]\(\]/\\OpP] # Mouth
    )"""
	
# Regex for non alphanumeric characters
regex_str = [
    r'<[^>]+>',  # HTML tags
    r"(?:[a-z][a-z\-_]+[a-z])",  # words with - and '
    r'(?:[\w_]+)',  # other words
    r'(?:\S)'  # anything else
]

In [13]:
# Prepare the dataset to remove to remove stop words by intializing the system
def initializeSystem():
    print("Preparing stop words.")
    stop = set(stopwords.words('english') + punctuation + ['rt', 'via', 'i\'m', 'us', 'it'])
    for x in stop:
        stop_words.append(stemmer.stem(lemmatiser.lemmatize(x, pos="v")))

In [14]:
# initializing NLTK functions for preprocessing
tokens_re = re.compile(r'(' + '|'.join(regex_str) + ')', re.VERBOSE | re.IGNORECASE)
emoticon_re = re.compile(r'^' + emoticons_str + '$', re.VERBOSE | re.IGNORECASE)
punctuation = list(string.punctuation)
stemmer = PorterStemmer()
lemmatiser = WordNetLemmatizer()
initializeSystem()

Preparing stop words.


In [15]:
print("Preprocess the Data")

# Tokenize will separate each word as tokens
def tokenize(s):
    return tokens_re.findall(s)

# This function removes non-alhanumeric characters, removes the emoticons if present and stems and lemmatize each word
def preprocessing(s, lowercase=True):
    s = re.sub('[^a-zA-Z0-9\n\.]', ' ', s)
    tokens = tokenize(s)
    if lowercase:
        tokens = [token if emoticon_re.search(token) else stemmer.stem(lemmatiser.lemmatize(token.lower(), pos="v")) for
                  token in tokens]
    return tokens


# Each row is broken down into space separated words or terms of list and sent for stemming and lemming. Here stop words are removed
def process_string(string):
    term_stop = [term for term in preprocessing(string) if term not in stop_words and len(str(term)) > 1]
    return term_stop

# This function is used to send each rows to preprocessing and then combines the result back to a single dataset
def preprocess_reviews(reviews):
    preprocessed_reviews = []
    for index, row in reviews.iterrows():
        temp = process_string(str(row["Review"]).lower())
        str1 = ' '.join(temp)
        temp = [str1]
        temp.append(row["Reviewer_Score"])
        preprocessed_reviews.append(temp)
    return preprocessed_reviews

train_reviews = preprocess_reviews(train_data_features_reviews)

Preprocess the Data


In [16]:
yx_lines = sc.parallelize(train_reviews)
train, test = yx_lines.randomSplit([0.6, 0.4])

target_rating = train.map(lambda x: x[1]).collect()
target_review = train.map(lambda x: x[0]).collect()

test_review = test.map(lambda x: x[0]).collect()
test_rating = test.map(lambda x: x[1]).collect()

In [17]:
# Function is used to send each row for preprocessing
def preprocess_reviews_lib(reviews):
    preprocessed_reviews_lib = []
    for a in reviews:
        preprocessed_reviews_lib.append(process_string(str(a).lower()))
    return preprocessed_reviews_lib

In [18]:
# Send training and test set reviews separately for preprocessing
train_reviews = preprocess_reviews_lib(target_review)


In [19]:
# Function to get decision attrubites
def get_decision_attributes(processed_reviews):
    to_count = []
    for a in processed_reviews:
        to_count.append(" ".join(a))
    str1 = ""
    for a in to_count:
        str1 += "".join(a)
    x = Counter(str1.split(" "))
    for (k, v) in x.most_common(min(500, len(x))):
        decision_attributes.append(k)

In [20]:
get_decision_attributes(train_reviews)

In [21]:
# Function to prepare sparse matrix
def prepare_sparse_matrix(processed_reviews):
    sparse_matrix = []
    for cr in processed_reviews:
        new_cr = [0] * len(decision_attributes)
        for word in cr:
            if word in decision_attributes:
                index = decision_attributes.index(word)
                new_cr[index] += 1
            else:
                pass
        sparse_matrix.append(new_cr)
    return sparse_matrix

In [22]:
# Create sparse matrix for the both and test and training dataset
train_sparse_matrix = prepare_sparse_matrix(train_reviews)
test_sparse_matrix = prepare_sparse_matrix(preprocess_reviews_lib(test_review))

In [23]:
data_features = pd.DataFrame(train_sparse_matrix, columns=decision_attributes)
test_data_features = pd.DataFrame(test_sparse_matrix, columns=decision_attributes)

In [24]:
# To fit the model built by the scikit learn library
def fitter(func_value):
    nb_func_value = str(func_value)
    nb_func_value = nb_func_value.partition("(")[0]
    func_value.fit(data_features, target_rating)
    s = f = 0
    predicted_rating = list(func_value.predict(test_data_features))
    for i in range(len(predicted_rating)):
        if predicted_rating[i] == test_rating[i]:
            s += 1
        else:
            f += 1
    print("Prediction rate by using " + nb_func_value + " is: " + str(float(s) / len(predicted_rating) * 100.0))

In [55]:
def pred_rat(s):
    user_review_matrix = prepare_sparse_matrix([process_string(str(s).lower())])
    user_review_features = pd.DataFrame(user_review_matrix, columns=decision_attributes)
    print(clf1.predict(user_review_matrix)[0])

In [25]:
# calculate multinomial naive bayes and fit the data to the model
clf1 = MultinomialNB()
fitter(clf1)

Prediction rate by using MultinomialNB is: 35.5298610169


In [54]:
pred_rat("awesome food great view")

10


In [53]:
pred_rat("bad food bad view")

7
